In [1]:
using Pkg
Pkg.update()

ipks = Pkg.installed()
pkgs = ["PyCall","HTML","HTTP","SQLite","Gumbo","Distributed"]
for p in pkgs
    !haskey(ipks,p) && Pkg.add(p)
end


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /Users/malcolm/.julia/registries/General — failed to fetch from repo
└ @ Pkg.API /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:157


  Updating git-repo `https://github.com/tonyhffong/Lint.jl.git`


Pkg.Types.PkgError: failed to fetch from https://github.com/tonyhffong/Lint.jl.git, error: GitError(Code:ERROR, Class:Net, curl error: Could not resolve host: github.com
)

# Chapter 11 - Networks and  Parallel Processing

In [1]:
using Sockets, Dates

In [2]:
s1 = UDPSocket();     # => UDPSocket(init)
s2 = UDPSocket();     # => UDPSocket(init)
(s1, s2)

(UDPSocket(init), UDPSocket(init))

In [3]:
bind(s1,ip"127.0.0.1",8011)

true

In [4]:
bind(s2,ip"127.0.0.1",8012)

true

In [5]:
now = string(Dates.now())
send(s2,ip"127.0.0.1",8011,string(Dates.now()))

In [6]:
(who, msg) = recvfrom(s1)
println(String(msg))

2019-03-14T13:45:58.912


In [7]:
close(s1);
close(s2);

---

## Working with the WEB

In [28]:
using HTTP
r = HTTP.request("GET", "http://amisllp.com")

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Date: Wed, 20 Mar 2019 22:21:43 GMT
Server: Apache
Last-Modified: Fri, 02 Nov 2018 09:07:33 GMT
Accept-Ranges: bytes
Content-Length: 5755
Content-Type: text/html

<!DOCTYPE html>
<html>
<head>
	<meta charset="utf-8">
    <meta name="Description" content="Elegance Theme">
    <meta name="author" content="CoffeeCup Software, Inc.">
    <meta name="Copyright" content="Copyright (c) 2011 CoffeeCup, all rights reserved.">
    <title>Amis Consulting LLP - About Us</title>
    <link rel="stylesheet" href="stylesheets/default.css" />
    <!--[if IE]>
    <script src="http://html5shim.googlecode.com/svn/trunk/html5.js"></script>
    <![endif]-->
</head>
<body>
    <section id="mainRight" class="clear">
         <header>
<img src="images/banner_home.png" width="960" height="140" alt="" title="" />
        </header>
		   
      <nav>
      <ul>
      <li><a href="index.html" class="selected">Home Page</a></li>
      <li><a href="about.html">About Us</a>

In [ ]:
### Cut and paste the following into a terminal tab
#=
using HTTP, Sockets
HTTP.listen(ip"127.0.0.1",8082) do http::HTTP.Stream
    @show HTTP.header(http, "Content-Type")
    @show http.message
    while !eof(http)
        println("body data: ", String(readavailable(http)))
    end
    HTTP.setstatus(http, 200)
    HTTP.setheader(http, "Content-Type" => "text/html")
    HTTP.startwrite(http)
    write(http, "<html><body>\n")
    write(http, "Is this a dagger?<br/>\n")
    write(http, "William Shakespeare<br/>\n")
    write(http,"</body></html>\n")
end
=#

---

In [31]:
HTTP.get("http://localhost:8082/")

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Content-Type: text/html
Transfer-Encoding: chunked

<html><body>
Is this a dagger?<br/>
William Shakespeare<br/>
</body></html>
"""

---

In [33]:
using SQLite
db = SQLite.DB("quotes.db")

SQLite.DB("quotes.db")

In [71]:
function getquote(db::SQLite.DB)
    sql = "select * from quotes where id =  (1 + abs(random() % 36))";
    res = SQLite.Query(db, sql) |> DataFrame
    author  = res[:author][1]
    if (author == "missing") author = "" end
    quoname = res[:quoname][1]
    (author, quoname)
end

(auth,quot) = getquote(db)
@printf "%s => %s" auth quot

Thomas Huxley => The great tragedy of science:  The slaying of a beautiful theory by an ugly fact

---

In [78]:
using HTTP, Gumbo
const JURL = "https://julialang.org"

function fetchpage(url)
  response = HTTP.get(url)
  if response.status == 200 && 
        parse(Int, Dict(response.headers)["Content-Length"]) > 0
    String(response.body)
  else
    ""
  end
end

function extractlinks(elem, links)
  if  isa(elem, HTMLElement) &&
      tag(elem) == :a &&
      in("href", collect(keys(attrs(elem))))
        url = getattr(elem, "href")
        startswith(url, "/") && length(url) > 1  && push!(links, url)
  end
  for child in children(elem)
    extractlinks(child, links)
  end
end

extractlinks (generic function with 1 method)

In [79]:
content = fetchpage(JURL)
jlinks = String[]

if ! isempty(content)
  dom = Gumbo.parsehtml(content)
  extractlinks(dom.root, jlinks)
end

display(unique(jlinks))

16-element Array{String,1}:
 "/downloads"                  
 "//docs.julialang.org"        
 "/blog/"                      
 "/community"                  
 "/learning"                   
 "/research"                   
 "/soc/ideas-page"             
 "/benchmarks/"                
 "//juliaobserver.com"         
 "/blog/2019/02/julia-entities"
 "/blog/2019/02/light-graphs"  
 "/blog/2019/01/fluxdiffeq"    
 "/blog"                       
 "/downloads/index.html"       
 "/downloads/oldreleases.html" 
 "/research/"                  

---

### Tasks

In [8]:
ta = Task(() -> let s = 0; sum(s += i for i in 1:100); println(s) end)

Task (runnable) @0x000000011c365210

In [9]:
istaskdone(ta)

false

In [10]:
schedule(ta)

5050


Task (done) @0x000000011c365210

---

In [11]:
# Define a Fibonnaci function
fib(n::Integer) = n < 3 ? 1 : fib(n-1) + fib(n-2)

fib (generic function with 1 method)

In [12]:
# ... and create an asynchronous task
chnl = Channel{Int}(1)
m = 15
@async for i=1:m
  put!(chnl, fib(i)) 
end

Task (runnable) @0x000000011bd79450

In [ ]:
# The task blocks until we take from the channel
for i in 1:m
  println(i, " => ", take!(chnl))
end

close(chnl);

In [13]:
# Tasks are iterable, ...
# ... we need to break out in order to close the task

chnl = Channel{Int}(1)
m = 20
@async for i=1:m
  put!(chnl, fib(i))
end

Task (runnable) @0x000000011bd798d0

In [14]:
# Take off the channel
# Bail out is x gets large
for p in chnl
    if p > 100
        break
    else
        println(p)
        yield()
    end
end

close(chnl)

1
1
2
3
5
8
13
21
34
55
89


---

### Needles and PI-(ns)

In [15]:
using Distributed

In [16]:
addprocs(3)   # Add three addition processors

3-element Array{Int64,1}:
 2
 3
 4

In [17]:
# Sequential version
function needles_seq(n::Integer)
  @assert n > 0
  k = 0
  for i = 1:n
    ρ = rand()
    ϕ = (rand() * π) -  π / 2   # angle at which needle falls
    xr = ρ + cos(ϕ)/2          # x-location of needle
    xl = ρ - cos(ϕ)/2
    # Count times when needle crosses either x == 0 or x == 1?
    k += (xr >= 1 || xl <= 0) ? 1 : 0
  end
  m = n - k
  return (n / k * 2)
end

needles_seq(1000)  # Compile and test the function, pretiming

3.053435114503817

In [18]:
# Parallel version

function needles_par(n)
  k = @distributed (+) for i = 1:n
    ρ = rand()
    ϕ = (rand() * π) - π / 2
    xr = ρ + cos(ϕ)/2
    xl = ρ - cos(ϕ)/2
    (xr >= 1 || xl <= 0) ? 1 : 0
  end
  m = n - k
  return(n / k * 2)
end

needles_par(250)    # Compile and test the function, pretiming
                    # Need less 25% trials as using 4 processors

3.125

In [19]:
# Now time the two approaches (make be effected by Jupyter)

@time needles_seq(100*10^6)

  4.040470 seconds (1.95 k allocations: 117.152 KiB)


3.141826296184395

In [20]:
@time needles_par(100*10^6)

  1.729999 seconds (347 allocations: 24.078 KiB)


3.1418380428045585

---

### Remote procedure calls and map-reduce

In [22]:
using DistributedArrays

In [23]:
# Add an extra processor and show the workers

addprocs(1)
for pid in workers()
    @show pid
end

pid = 2
pid = 3
pid = 4
pid = 5


In [24]:
# Run this on PID = 2
r1 = remotecall(x -> x^2, 2, π)

Future(2, 1, 18, nothing)

In [25]:
# ... and get the result
fetch(r1)

9.869604401089358

In [26]:
# Remote call/fetch can be do in a single step
remotecall_fetch(sin, 5, π/4)

0.7071067811865475

In [27]:
# Let's workout some factorials on different processors
# Each one needs to 'know' the fac() definition
@everywhere fac(n::Integer) = (n < 2) ? 1 : n*fac(n-1)

In [28]:
# Spawn the task over all the workers
r = [@spawnat w fac(3 + 2*w) for w in workers()]

4-element Array{Future,1}:
 Future(2, 1, 29, nothing)
 Future(3, 1, 30, nothing)
 Future(4, 1, 31, nothing)
 Future(5, 1, 32, nothing)

In [29]:
# ... and return all the results
for id in r
    @show id, fetch(id)
end

(id, fetch(id)) = (Future(2, 1, 29, Some(5040)), 5040)
(id, fetch(id)) = (Future(3, 1, 30, Some(362880)), 362880)
(id, fetch(id)) = (Future(4, 1, 31, Some(39916800)), 39916800)
(id, fetch(id)) = (Future(5, 1, 32, Some(6227020800)), 6227020800)


In [30]:
# Just as a check
fac(7),fac(9),fac(11),fac(13)

(5040, 362880, 39916800, 6227020800)

In [31]:
@everywhere using DistributedArrays, Statistics

In [32]:
# Create a DA, ~ Normal(0.0,1.0)
#
da = drandn(10,10,10,10);

In [33]:
fieldnames(typeof(da))

(:id, :dims, :pids, :indices, :cuts, :localpart, :release)

In [34]:
da.dims

(10, 10, 10, 10)

In [35]:
da.indices

1×1×2×2 Array{NTuple{4,UnitRange{Int64}},4}:
[:, :, 1, 1] =
 (1:10, 1:10, 1:5, 1:5)

[:, :, 2, 1] =
 (1:10, 1:10, 6:10, 1:5)

[:, :, 1, 2] =
 (1:10, 1:10, 1:5, 6:10)

[:, :, 2, 2] =
 (1:10, 1:10, 6:10, 6:10)

In [36]:
for ip in procs(da)
    rp = @spawnat ip mean(localpart(da))
    @show fetch(rp)
end

fetch(rp) = 0.008624726127582843
fetch(rp) = 0.004794672601544152
fetch(rp) = -0.02017443022421869
fetch(rp) = -0.0013743698123322929


In [37]:
# Calculate the mean (~ 0.0)

using Printf
let 
  μ = 0.0
  for ip in procs(da)
    rp = @spawnat ip mean(localpart(da))
    μ += fetch(rp)
  end
  @printf "Mean = %.6f\n" μ/length(workers())
end

Mean = -0.002032


In [38]:
# ...  and the standard deviation (~ 1.0)
let 
  σ = 0.0
  for ip in procs(da)
    rp = @spawnat ip std(localpart(da))
    σ += fetch(rp)
  end
  @printf "Standard Deviation = %.6f\n" σ/length(workers())
end

Standard Deviation = 0.987293


---

In [41]:
# Converting from a local array to a distributed is very simple
# using the distribute() routine 

using Distributed
addprocs(4)

4-element Array{Int64,1}:
 6
 7
 8
 9

In [43]:
# Make sure the remote processors have a copy of the packages
@everywhere using Distributed, DistributedArrays, SpecialFunctions

aa = [rand() for i = 1:100,j = 1:100];
da = distribute(aa);

In [44]:
# Check da is a Distributed Array
(da.dims,da.indices)

((100, 100), Tuple{UnitRange{Int64},UnitRange{Int64}}[(1:50, 1:25) (1:50, 26:50) (1:50, 51:75) (1:50, 76:100); (51:100, 1:25) (51:100, 26:50) (51:100, 51:75) (51:100, 76:100)])

In [45]:
# This mapping works
@elapsed bb = map(x -> abs(bessely0(x)), aa)

0.09991644

In [46]:
# ... and also so does this just a little more slowly!
@elapsed db = map(x -> abs(bessely0(x)), da)

4.616189086

In [47]:
# Check random element are same
i = rand(1:100); j = rand(1:100)
bb[i,j] == db[i,j]

true

In [52]:
abs(sum(bb) - sum(db))

9.094947017729282e-13